## Setup


In [ ]:
import requests
import json
import base64
import brikasutils as bu
import test_utils as tu
import random
from aiohttp import ClientTimeout  # noqa: F401
from dotenv import load_dotenv
from typing import Dict, List
import os
from pathlib import Path
from aiohttp import FormData

from faker import Faker
fake = Faker()
Faker.seed("MEALMEAP")

import importlib
importlib.reload(bu)
importlib.reload(tu)

from brikasutils import RequestTemplate as RT

load_dotenv(override=True)
ADMIN_API_KEY = os.getenv('ADMIN_API_KEY') or 'undefined'

HEADERS_CONTENT_JSON = {'Content-Type': 'application/json'}

# Endpoint Base URLs

# 1. PRODUCTION (Railway)
# BASE_URL_AUTH = 'https://mealmap-production.up.railway.app/auth'
# BASE_URL_FUNCTIONS = 'https://mealmap-production.up.railway.app'

# 2. LOCAL (Development)
BASE_URL_AUTH = 'http://localhost:8000/auth'
BASE_URL_FUNCTIONS = 'http://localhost:8000'

In [ ]:
## send simple GET to /test.
response = requests.get(f"{BASE_URL_FUNCTIONS}/test")
print("Response Status Code:", response.status_code)
print("Response Body:", response.json())


## Admin

In [ ]:
# Recompute all meal features (Backfill)
# Run this once to populate ComputedMealFeatures for existing meals
DO_RUN = False
if DO_RUN:
    headers = {"X-Admin-Key": ADMIN_API_KEY}
    response = requests.post(f"{BASE_URL_FUNCTIONS}/admin/recompute-features", headers=headers)
    print("Recompute Status:", response.status_code)
    print("Response:", json.dumps(response.json(), indent=2))

## Users


### Setup Users


In [ ]:
MIME_TYPE = 'image/jpeg'

IMAGE_SQ_PERSON_1 = 'img/test-img-sq-person-1.jpg'
IMAGE_SQ_PERSON_2 = 'img/test-img-sq-person-2.jpg'
IMAGE_SQ_PERSON_3 = 'img/test-img-sq-person-3.jpg'
IMAGE_SQ_PERSON_4 = 'img/test-img-sq-person-4.jpg'
IMAGE_NONSQ_PERSON_1 = 'img/test-img-nonsq-person-1.jpg'
IMAGE_NONSQ_PERSON_2 = 'img/test-img-nonsq-person-2.jpg'


IMAGE_SQ_GROUP_DORM = 'img/test-img-sq-group-dorm.jpg'
IMAGE_SQ_GROUP_DTU = 'img/test-img-sq-group-dtu.jpg'
IMAGE_SQ_GROUP_LOGO = 'img/test-img-sq-group-logo.jpg'


users = [
    {
        "email": "airidas.brikas@gmail.com",
        "password": "1234567",
        "first_name": "Airidas",
        "last_name": "Brikas",
        "image": IMAGE_SQ_PERSON_1,
        # "token": str
        # "id": str
        # "groups": ["..."]

    },
    {
        "email": "rares@t.com",
        "password": "1234567",
        "first_name": "Rares",
        "last_name": "Diaconescu",
        "image": IMAGE_SQ_PERSON_2,
    },
    {
        "email": "anna@t.com",
        "password": "1234567",
        "first_name": "Anna",
        "last_name": "Kowalska",
        "image": IMAGE_SQ_PERSON_3,
    },
    {
        "email": "elias@t.com",
        "password": "1234567",
        "first_name": "Elias",
        "last_name": "Lunoe",
        "image": IMAGE_SQ_PERSON_4
        # No image 
    },
    {
        "email": "david@t.com",
        "password": "1234567",
        "first_name": "David",
        "last_name": "Jaic",
        "image": IMAGE_SQ_PERSON_3,
    },
    {
        "email": "george@t.com",
        "password": "1234567",
        "first_name": "George",
        "last_name": "Andrei",
        "image": IMAGE_SQ_PERSON_4,
    },


]

# Apply defaults
def apply_user_defaults():
    for user in users:
        if "send_welcome_email" not in user:
            user["send_welcome_email"] = False # type: ignore
        if "image" not in user:
            user["image"] = None # type: ignore
apply_user_defaults()

##### _Add additional generated users_


In [ ]:
# Generate X amount of groups using faker
NUM_USERS = 0  # Set the number of groups you want to generate
BATCH_ID = "gen-users-1"

add_users = []
for i in range(NUM_USERS):
    add_users.append({
        "email": fake.email(),
        "password": "1234567",
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "batch_id": BATCH_ID,
    })

# Add generated groups to the main groups list and apply defaults
users.extend(add_users)
apply_user_defaults()
print(f"Added {NUM_USERS} generated users.")

### Register New users


In [ ]:
ONLY_NON_EXISTING = True

SEND_EMAIL = False
rts = []
for user in users:
    if ONLY_NON_EXISTING and 'id' in user and user["id"] != None:
        continue

    def callback(response: bu.CustomResponse, user=user):        
        if response.status == 200:
            print(f"{response.status}: {user['email']}")
            r = response.json()
            if r != None:
                user['token'] = r["access_token"]
                user['id'] = r["user_id"]
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_AUTH}/register',
            json={
                "email": user['email'],
                "password": user['password'],
                "send_email": True if user['send_welcome_email'] else SEND_EMAIL,
                "test_id": user['batch_id'] if 'batch_id' in user else None,
            },
            timeout=60,
            headers=HEADERS_CONTENT_JSON,
            callback=callback,
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=5)

### Log in Users


In [ ]:
ONLY_NON_LOGGED_IN = False
AUTH_ENDPOINT = f'{BASE_URL_AUTH}/token'

rts = []

for user in users:
    if ONLY_NON_LOGGED_IN and 'token' in user and user['token'] != None:
        continue

    if "email" in user and "password" in user:
        def callback(response: bu.CustomResponse, user=user):
            if response.status == 200:
                print(f"{response.status}: {user['email']}")
                r = response.json()
                if r != None:
                    user['token'] = r["access_token"]
                    user['id'] = r["user_id"]
            else:
                print(f"{response.status}: {user['email']}, response: {response.text}")

        rts.append(
            RT(
                method="post",
                url=AUTH_ENDPOINT,
                json={
                    "email": user['email'],
                    "password": user['password'],
                },
                timeout=30,
                headers=HEADERS_CONTENT_JSON,
                callback=callback
            )
        )

responses = bu.run_async_requests(rts, verbose=False)

### Me

In [ ]:
rts = []
for user in users:
    if "token" not in user or user['token'] == None:
        continue

    def callback(response: bu.CustomResponse, user=user):
        if response.status == 200:
            r = response.json()
            print(f"{response.status}: {user['email']} -> {r}")
            if r:
                user.update(r)
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users/me',
            headers=tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Update User


In [ ]:
only_users = [
    # tu.get_user_from_email("petyo@t.com", users)['id'],
]

rts = []
for user in users:

    if len(only_users) > 0 and user['id'] not in only_users:
        continue

    def callback(response: bu.CustomResponse, user=user):
        print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")


    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/{user["id"]}',
            json={
                "first_name": user['first_name'],
                "last_name": user['last_name'],
                # "image": tu.encode_base64_image_or_none(user['image']),
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=10)

### Change Password


In [ ]:
queries = [
    # {
    #     "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
    #     "current_password": "1234567890",
    #     "new_password": "1234567",
    # },
    # {
    #     "action_caller": tu.get_user_from_email("rares@t.com", users),
    #     "current_password": "1234567",
    #     "new_password": "12345678",
    # }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status == 200:
            r = response.json()
            print(f"{response.status}: {tu.emailOrPhone(user)}. Updating local records.")
            user['password'] = query['new_password']
        else:
            print(f"{response.status}: {tu.emailOrPhone(user)}, response: {response.text}")
    
    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/users/me/change-password',
            json={
                "current_password": query['current_password'],
                "new_password": query['new_password'],
            },
            timeout=30,
            headers= tu.auth_json_headers(user['token']),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Search Users


In [ ]:
queries = [
    {
        "action_caller": { "email": "david@t.com"},
        "query": "rar",
    }
]

rts = []
for query in queries:
    user = tu.get_user_from_email(query['action_caller']['email'], users)

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {len(r['results'])} results")
                for res in r['results']:
                    print(f" - {res['first_name']} {res['last_name']} ({res['email']})")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users',
            params={
                "q": query["query"],
            },
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Get User


In [ ]:
queries = [
    {
        "action_caller_id": tu.get_user_from_email("rares@t.com", users)['id'],
        "user_id": tu.get_user_from_email("airidas.brikas@gmail.com", users)["id"],
    },
]

rts = []
for query in queries:
    user = tu.get_user_from_id(query['action_caller_id'], users)

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {query["user_id"]},")
                print(json.dumps(r, indent=2))
        else:
            print(f"{response.status}: {user['email']} -> {query["user_id"]}, response: {response.text}")

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/users/{query["user_id"]}',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Delete User


In [ ]:
# Delete user account
queries = [
    # {
    #     "action_caller": tu.get_user_from_email("patryk@t.com", users),
    # }
]

rts = []
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user):
        if response.status_code == 200:
            print(f"{response.status}: {user['email']} - Account deleted successfully")
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="delete",
            url=f'{BASE_URL_FUNCTIONS}/users/me',
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

## Places

### Setup Places

In [ ]:
import test_data
importlib.reload(test_data)
places_data = test_data.get_places()

MIME_TYPE = 'image/jpeg'

user_pool = [
    tu.get_user_from_email("airidas.brikas@gmail.com", users),
    tu.get_user_from_email("rares@t.com", users),
    tu.get_user_from_email("anna@t.com", users),
    tu.get_user_from_email("elias@t.com", users),
]

places = []
for place_data in places_data:
    # Randomly assign a user to create each place
    user = random.choice(user_pool)

    place = {
        "action_caller": {"email": user['email']},
        "name": place_data["name"],
        "address": place_data["address"],
        "latitude": place_data["latitude"],
        "longitude": place_data["longitude"],
        "images": place_data.get("images", []),  # Get images from test_data if provided
        "test_id": place_data["test_id"],
        "cuisine": place_data["cuisine"], 
        # "id": "..." will be populated after creation
    }
    places.append(place)

# Apply defaults
def apply_place_defaults():
    for place in places:
        if "images" not in place:
            place["images"] = []

apply_place_defaults()

print(f"Loaded {len(places)} places from test_data.py")
print(f"Places distributed among {len(user_pool)} users")

# Print statistics about images
places_with_images = [p for p in places if p["images"]]
print(f"{len(places_with_images)} places have images")

### Create places

In [ ]:
rts = []
ONLY_NON_EXISTING = True

for place in places:
    if "id" in place and ONLY_NON_EXISTING:
        print(f"Place {place['test_id']} already exists, skipping")
        continue

    user = tu.get_user_from_email(place['action_caller']['email'], users)

    form = FormData()
    form.add_field('name', place['name'])
    form.add_field('latitude', str(place['latitude']))
    form.add_field('longitude', str(place['longitude']))
    
    if 'address' in place and place['address']:
        form.add_field('address', place['address'])

    if 'cuisine' in place and place['cuisine']:
        form.add_field('cuisine', place['cuisine'])

    if 'images' in place:
        for img_path in place['images']:
            form.add_field(
                'images',
                open(img_path, 'rb'),
                filename=Path(img_path).name,
                content_type="image/jpeg"
            )
    
    if 'test_id' in place and place['test_id']:
        form.add_field('test_id', place['test_id'])
            

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {place['name']} by {user['email']} -> {r['id']}")
                place['id'] = r['id']
        else:
            print(f"{response.status}: {place['name']} & {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/places',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)
tu.print_response_code_summary(responses)

### List places

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "lat": 36.373,
        "lng": 127.367,
        "radius_meters": 5000,
        "name": None,
        "sort_by": "distance",
        "sort_order": "asc",
    },
    # {
    #     "action_caller": tu.get_user_from_email("rares@t.com", users),
    #     "lat": 36.3735,
    #     "lng": 127.3675,
    #     "radius_meters": 10000,
    #     "name": "pizza",
    #     "sort_by": "distance",
    #     "sort_order": "asc",
    # },
    
]

rts = []

for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {len(r['results'])} places found")
                for place in r['results']:
                    print(f"  - {place['name']} ({place['distance_meters']:.0f}m)")
                    
                if r['current_page'] < r['total_pages']:
                    print(f"Fetching page {r['current_page'] + 1}/{r['total_pages']}...")
                    return response.get_request_template_copy_with_updated_params(
                        params_update={
                            "page": r['current_page'] + 1,
                        }
                    )
        else:
            print(f"{response.status}: {user['email']}, response: {response.text}")

    params = {
        "lat": query['lat'],
        "long": query['lng'],
        "radius_meters": query['radius_meters'],
        "sort_by": query['sort_by'],
        "sort_order": query['sort_order'],
    }
    if query['name']:
        params['name'] = query['name']

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/places',
            params=params,
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

# Extract all discovered places from successful responses
stats_updated, stats_added = 0, 0
for response in responses:
    if response.status_code == 200:
        r = response.json()
        if r and 'results' in r:
            for place_data in r['results']:
                _, was_added = tu.update_or_add_object(place_data, places, identifier_key='test_id')
                if was_added:
                    stats_added += 1
                else:
                    stats_updated += 1

print(f"\nLocal records: Updated {stats_updated} and Added {stats_added} new. (total {len(places)})")

### Get place details 

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "place_test_id": "subway-kaist",
        "lat": 36.3682,
        "lng": 127.3685,
    },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "place_test_id": "pizza-school",
        "lat": None,
        "lng": None,
    },
]

rts = []

for query in queries:
    user = query['action_caller']
    place = tu.get_object_from_test_id(query['place_test_id'], places)

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {place['name']}")
                print(json.dumps(r, indent=2))
                place.update(r)
        else:
            print(f"{response.status}: {user['email']} & {place['name']}, response: {response.text}")

    params = {}
    if query['lat'] is not None and query['lng'] is not None:
        params['lat'] = query['lat']
        params['long'] = query['lng']

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/places/{place["id"]}',
            params=params if params else None, # type: ignore
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Update place

In [ ]:
place_updates = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "place_test_id": "subway-kaist",
        "form_data_str": {
            "name": "Subway KAIST - Updated",
            "address": "291 Daehak-rooooo, Yuseong-gu, Daejeon, South Korea",
        },
        "form_data_files": {
            "add_images": [
                # IMAGE_PLACE_3,
            ],
        },
        "remove_image_ids": [],
    },
    {
        "action_caller": tu.get_user_from_email("anna@t.com", users),
        "place_test_id": "pizza-school",
        "form_data_str": {
            "address": "123 New Address St, City, Country",
        },
        "form_data_files": {
            "add_images": [
                # IMAGE_PLACE_1,
                # IMAGE_PLACE_2,
            ],
        },
        "remove_image_ids": [],
    },
]

In [ ]:
# Update just the cuisine from existing places var
place_updates = []

for place in places:
    place_updates.append(
        {
            "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
            "place_test_id": place['test_id'],
            "form_data_str": {
                "cuisine": place['cuisine'], 
            },
            "form_data_files": {},
            "remove_image_ids": [],
        }
    ) 
print(f"Prepared {len(place_updates)} place updates.")

In [ ]:
rts = []

for upd in place_updates:
    user = tu.get_user_from_email(upd['action_caller']['email'], users)
    place = tu.get_object_from_test_id(upd['place_test_id'], places)


    form = FormData()

    if 'form_data_str' in upd:
        for field, value in upd['form_data_str'].items():
            if isinstance(value, list):
                for v in value:
                    form.add_field(field, str(v))
            else:
                form.add_field(field, str(value))

    if 'form_data_files' in upd:
        for field, files in upd['form_data_files'].items():
            for file_path in files:
                form.add_field(
                    field,
                    open(file_path, "rb"),
                    filename=Path(file_path).name,
                    content_type="image/jpeg"
                )

    if upd.get('remove_image_ids'):
        remove_ids_str = ','.join(str(img_id) for img_id in upd['remove_image_ids'])
        form.add_field('remove_image_ids', remove_ids_str)

    def callback(response: bu.CustomResponse, user=user, place=place):
        if response.status_code == 200:
            r = response.json()
            print(f"{response.status}: {user['email']} -> {place['name']}")
            print(json.dumps(r, indent=2))
        else:
            if response.json() is not None:
                print(f"{response.status}: {user['email']} & {place['name']}")
                print(json.dumps(response.json(), indent=2))
            else:
                print(f"{response.status}: {user['email']} & {place['name']}, response: {response.text}")

    rts.append(
        RT(
            method="put",
            url=f"{BASE_URL_FUNCTIONS}/places/{place['id']}",
            data=form,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=60,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

## Meals

### Setup Meals

In [ ]:
import test_data
importlib.reload(test_data)
meals_data = test_data.get_meals()

meals = []
for meal_data in meals_data:
    # Randomly assign a user to create each meal (or use place owner if we tracked it, but random is fine for now)
    user = random.choice(user_pool)

    meal = {
        "action_caller": {"email": user['email']},
        "place_test_id": meal_data["place_test_id"],
        "name": meal_data["name"],
        "test_id": meal_data["test_id"],
        "images": meal_data.get("images", []),
        # "id": "..." will be populated after creation
    }
    meals.append(meal)

print(f"Loaded {len(meals)} meals from test_data.py")
print(f"{len([m for m in meals if m["images"]])} meals have images")

### Create Meals

In [ ]:
rts = []
ONLY_NON_EXISTING = True

# First, resolve place_id from place_test_id
for meal in meals:
    place = tu.get_object_from_test_id(meal["place_test_id"], places)
    meal["place_id"] = place["id"]

for meal in meals:
    if "id" in meal and ONLY_NON_EXISTING:
        print(f"Meal {meal['test_id']} already exists, skipping")
        continue

    user = tu.get_user_from_email(meal['action_caller']['email'], users)

    form = FormData()
    form.add_field('place_id', str(meal['place_id']))
    form.add_field('name', meal['name'])
    if meal.get('price'):
        form.add_field('price', str(meal['price']))
    if meal.get('test_id'):
        form.add_field('test_id', meal['test_id'])

    if meal.get('images'):
        for img_path in meal['images']:
            form.add_field(
                'images',
                open(img_path, 'rb'),
                filename=Path(img_path).name,
                content_type="image/jpeg"
            )

    def callback(response: bu.CustomResponse, user=user, meal=meal):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {meal['name']} by {user['email']} -> {r['id']}")
                meal['id'] = r['id']
        else:
            print(f"{response.status}: {meal['name']} & {user['email']}, response: {response.text}")

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/meals',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=5)

# Response code summary
tu.print_response_code_summary(responses)

### Get Meals

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "description": "All meals within 5km radius",
        "params": {
            "lat": 36.373,
            "lng": 127.367,
            "radius_m": 5000,
            "sort_by": "distance",
            "sort_order": "asc",
        }
    },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "description": "Search for 'pizza'",
        "params": {
            "name": "pizza",
            "sort_by": "rating",
            "sort_order": "desc",
        }
    }
]

rts = []

for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"\n{'='*70}")
                print(f"Query: {query['description']}")
                print(f"User: {user['email']}")
                print(f"Status: {response.status}")
                print(f"Results: {len(r['results'])} of {r['total_items']} total")
                print(f"{'='*70}")
                
                for idx, meal in enumerate(r['results'][:5], 1):
                    distance_info = f" ({meal['distance_meters']:.0f}m)" if meal.get('distance_meters') else ""
                    price_info = f" - ₩{meal['price']:.0f}" if meal.get('price') else ""
                    rating_info = f" - {meal['avg_rating']:.1f}★ ({meal['review_count']})" if meal.get('avg_rating') else " - New"
                    
                    print(f"\n{idx}. {meal['name']}{rating_info}")
                    print(f"   @ {meal['place_name']}{distance_info}")
                    print(f"   {price_info}")
                    
                    # Show tags
                    tags = []
                    meal_tags = meal.get('tags', {})
                    for k, v in meal_tags.items():
                        if v == 'yes':
                            tags.append(k.replace('is_', '').replace('_', ' ').title())
                    if tags:
                        print(f"   Tags: {', '.join(tags)}")

                if len(r['results']) > 5:
                    print(f"\n   ... and {len(r['results']) - 5} more")
        else:
            print(f"\n❌ {response.status}: {user['email']} - {query['description']}")
            print(f"   Response: {response.text}")

    params = query['params'].copy()
    if 'lng' in params:
        params['long'] = params.pop('lng')

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/meals',
            params=params,
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)
tu.print_response_code_summary(responses)

### Get Meal

In [ ]:
queries = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "meal_test_id": "pepperoni-pizza",
        "lat": 36.373,
        "lng": 127.367,
    }
]

rts = []

for query in queries:
    user = query['action_caller']
    meal = tu.get_object_from_test_id(query['meal_test_id'], meals)

    def callback(response: bu.CustomResponse, user=user, meal=meal):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} -> {meal['name']}")
                print(json.dumps(r, indent=2))
        else:
            print(f"{response.status}: {user['email']} & {meal['name']}, response: {response.text}")

    params = {}
    if 'lat' in query and 'lng' in query:
        params['lat'] = query['lat']
        params['long'] = query['lng']

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/meals/{meal["id"]}',
            params=params,
            timeout=10,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Update Meal

In [ ]:
meal_updates = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "meal_test_id": "pepperoni-pizza",
        "form_data": {
            "name": "Pepperoni Pizza",
            "price": "13500",
        }
    }
]

rts = []

for upd in meal_updates:
    user = upd['action_caller']
    meal = tu.get_object_from_test_id(upd['meal_test_id'], meals)

    form = FormData()
    for field, value in upd['form_data'].items():
        form.add_field(field, str(value))

    def callback(response: bu.CustomResponse, user=user, meal=meal):
        if response.status_code == 200:
            r = response.json()
            print(f"{response.status}: {user['email']} -> {meal['name']}")
            print(json.dumps(r, indent=2))
        else:
            print(f"{response.status}: {user['email']} & {meal['name']}, response: {response.text}")

    rts.append(
        RT(
            method="put",
            url=f"{BASE_URL_FUNCTIONS}/meals/{meal['id']}",
            data=form,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=30,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

### Delete Meal

In [ ]:
meals_to_delete = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "meal_test_id": "cheese-pizza",
    }
]

rts = []

for query in meals_to_delete:
    user = query['action_caller']
    try:
        meal = tu.get_object_from_test_id(query['meal_test_id'], meals)
    except ValueError:
        print(f"Meal {query['meal_test_id']} not found in local list, skipping delete")
        continue

    def callback(response: bu.CustomResponse, user=user, meal=meal):
        if response.status_code == 200:
            if response.json_data is not None:
                print(f"{response.status}: {user['email']} -> {response.json_data["message"]}")
            else:
                print(f"{response.status}: {user['email']} -> Deleted {meal['name']}, but no message returned")
            # if meal in meals:
            #     meals.remove(meal)
        else:
            print(f"{response.status}: {user['email']} failed to delete {meal['name']}")
            print(f"   Response: {response.text}")

    rts.append(
        RT(
            method="delete",
            url=f"{BASE_URL_FUNCTIONS}/meals/{meal['id']}",
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=30,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

## Reviews

### Setup Reviews

In [ ]:
import test_data
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter

importlib.reload(test_data)
reviews_data = test_data.get_reviews()

# Create user pool (same as places)
user_pool = [
    tu.get_user_from_email("airidas.brikas@gmail.com", users),
    tu.get_user_from_email("rares@t.com", users),
    tu.get_user_from_email("anna@t.com", users),
    tu.get_user_from_email("elias@t.com", users),
    tu.get_user_from_email("david@t.com", users),
    tu.get_user_from_email("george@t.com", users),
]

# Distribute reviews among users with normal distribution
np.random.seed(42)  # For reproducibility
reviews = []

for review_data in reviews_data:
    # Randomly assign a user following normal distribution
    user_idx = int(np.clip(np.random.normal(len(user_pool)/2, len(user_pool)/4), 0, len(user_pool)-1))
    user = user_pool[user_idx]
    
    review = {
        "test_id": review_data["test_id"],
        "action_caller": {"email": user['email']},
        "meal_test_id": review_data["meal_test_id"],
        "fields": {
            "rating": review_data["rating"],
            "text": review_data.get("text"),
            "waiting_time_minutes": review_data.get("waiting_time_minutes"),
            "test_id": review_data.get("test_id"),
            "price": review_data.get("price"),
            "is_vegan": review_data["is_vegan"],
            "is_halal": review_data["is_halal"],
            "is_vegetarian": review_data["is_vegetarian"],
            "is_spicy": review_data["is_spicy"],
            "is_gluten_free": review_data["is_gluten_free"],
            "is_dairy_free": review_data["is_dairy_free"],
            "is_nut_free": review_data["is_nut_free"],
        },
        "images": review_data.get("images", []),
        # "id": "..." will be populated after creation
    }
    reviews.append(review)

print(f"Loaded {len(reviews)} reviews from test_data.py")
print(f"Reviews distributed among {len(user_pool)} users")

# Print statistics about images
reviews_with_images = [r for r in reviews if r["images"]]
print(f"{len(reviews_with_images)} reviews have images")

#### Add reviews that don't have meal_id (so it should be auto created)

In [ ]:
new_reviews = [
    {
        "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
        "fields": {
            "place_id": tu.get_object_from_test_id("kaist-duck-pond-cafe", places)["id"],
            "meal_name": "Banana Smoothie",
            "rating": 5,
            "text": "Super nice!",
            "test_id": "banana-smoothie-1",
            "waiting_time_minutes": 5,
            "price": 5000,
            "is_vegan": "yes",
            "is_halal": "unspecified",
            "is_vegetarian": "yes",
            "is_spicy": "no",
            "is_gluten_free": "unspecified",
            "is_dairy_free": "unspecified",
            "is_nut_free": "no",
        },
        "images": []
    },
]

reviews.extend(new_reviews)

In [ ]:
# Analyze review statistics with distribution graphs
import test_extra
importlib.reload(test_extra)
# test_extra.analyze_review_statistics(reviews, places)

In [ ]:
reviews

### Create Reviews

In [ ]:
ONLY_NON_EXISTING = True

rts = []
for review in reviews:
    if "id" in review and ONLY_NON_EXISTING:
        print(f"Review for {review['test_id']} already exists, skipping")
        continue
    
    # Resolve meal_id from meal_test_id
    if "meal_name" not in review["fields"]:
        # Means meal was created earlier
        meal = tu.get_object_from_test_id(review["meal_test_id"], meals)
        review["fields"]["meal_id"] = meal["id"]

    user = tu.get_user_from_email(review['action_caller']['email'], users)
    fields = review['fields']

    form = FormData()
    
    if 'meal_id' in fields:
        form.add_field('meal_id', str(fields['meal_id']))
    elif 'meal_name' in fields and 'place_id' in fields:
        form.add_field('meal_name', fields['meal_name'])
        form.add_field('place_id', str(fields['place_id']))
    else:
        raise ValueError("Either meal_id or both meal_name and place_id must be provided in review fields")
    
    form.add_field('rating', str(fields['rating']))
    
    if fields.get('text'):
        form.add_field('text', fields['text'])
    if fields.get('waiting_time_minutes') is not None:
        form.add_field('waiting_time_minutes', str(fields['waiting_time_minutes']))
    if fields.get('test_id') is not None:
        form.add_field('test_id', fields['test_id'])
    if fields.get('price') is not None:
        form.add_field('price', str(fields['price']))
    if fields.get("meal_name"):
        form.add_field('meal_name', fields['meal_name'])
    if fields.get("place_id"):
        form.add_field('place_id', str(fields['place_id']))
        
    
    # Add dietary tags
    form.add_field('is_vegan', fields['is_vegan'])
    form.add_field('is_halal', fields['is_halal'])
    form.add_field('is_vegetarian', fields['is_vegetarian'])
    form.add_field('is_spicy', fields['is_spicy'])
    form.add_field('is_gluten_free', fields['is_gluten_free'])
    form.add_field('is_dairy_free', fields['is_dairy_free'])
    form.add_field('is_nut_free', fields['is_nut_free'])

    # Add images if they exist
    if 'images' in review and review['images']:
        for img_path in review['images']:
            form.add_field(
                'images',
                open(img_path, 'rb'),
                filename=Path(img_path).name,
                content_type="image/jpeg"
            )

    def callback(response: bu.CustomResponse, user=user, review=review):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                img_info = f" ({len(review['images'])} images)" if review.get('images') else ""
                if "test_id" not in review:
                    review['test_id'] = 'test_id', 'N/A'
                print(f"{response.status}: {review['test_id']} by {user['email']}{img_info} -> {r['id']}")
                review['id'] = r['id']
            
            
        else:
            print(f"{response.status}: {review['test_id']} & {user['email']}, response: {response.text}")
            

    rts.append(
        RT(
            method="post",
            url=f'{BASE_URL_FUNCTIONS}/reviews',
            data=form,
            timeout=30,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)
tu.print_response_code_summary(responses)

### Get Reviews

In [ ]:
import test_extra
importlib.reload(test_extra)

queries = [
    # {
    #     "action_caller": tu.get_user_from_email("airidas.brikas@gmail.com", users),
    #     "description": "All reviews within 2km radius, sorted by distance",
    #     "params": {
    #         "lat": 36.373,
    #         "lng": 127.367,
    #         "radius_m": 2000,
    #         "sort_by": "distance",
    #         "sort_order": "asc",
    #     }
    # },
    {
        "action_caller": tu.get_user_from_email("rares@t.com", users),
        "description": "All review for pepperoni-pizza meal",
        "params": {
            "meal_id": tu.get_object_from_test_id("pepperoni-pizza", meals)["id"],
        }
    },
    # {
    #     "action_caller": tu.get_user_from_email("rares@t.com", users),
    #     "description": "All reviews for Pizza School (via Place ID)",
    #     "params": {
    #         "place_id": tu.get_object_from_test_id("pizza-school", places)["id"],
    #         "sort_by": "rating",
    #         "sort_order": "desc",
    #     }
    # },
    # {
    #     "action_caller": tu.get_user_from_email("anna@t.com", users),
    #     "description": "Search reviews mentioning 'spicy' or 'crispy'",
    #     "params": {
    #         "text": "crispy",
    #         "sort_by": "created_at",
    #         "sort_order": "desc",
    #     }
    # },
    # {
    #     "action_caller": tu.get_user_from_email("anna@t.com", users),
    #     "description": "Recent reviews (no filters, default sort)",
    #     "params": {

    #     }
    # },
    # *test_extra.extra_get_reviews_test_queries(users)

]

rts = []

print(f"Running {len(queries)} review queries...\n")
for query in queries:
    user = query['action_caller']

    def callback(response: bu.CustomResponse, user=user, query=query):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"\n{'='*70}")
                print(f"Query: {query['description']}")
                print(f"User: {user['email']}")
                print(f"Status: {response.status}")
                print(f"Results: {len(r['results'])} of {r['total_items']} total")
                print(f"{'='*70}")
                
                for idx, review in enumerate(r['results'][:5], 1):  # Show first 5
                    distance_info = f" ({review['distance_meters']:.0f}m)" if review.get('distance_meters') else ""
                    price_info = f" - ₩{review['price']:.0f}" if review.get('price') else ""
                    wait_info = f" - {review['waiting_time_minutes']}min wait" if review.get('waiting_time_minutes') else ""
                    
                    print(f"\n{idx}. {review['meal_name']} - {review['rating']}★")
                    print(f"   @ {review['place']['name']}{distance_info}")
                    print(f"   By: {review['user']['first_name']} {review['user']['last_name']}")
                    
                    # Show dietary tags if any are "yes"
                    tags = []
                    # Tags are now nested in 'tags' object
                    review_tags = review.get('tags', {})
                    if review_tags.get('is_vegan') == 'yes':
                        tags.append('Vegan')
                    if review_tags.get('is_vegetarian') == 'yes':
                        tags.append('Vegetarian')
                    if review_tags.get('is_spicy') == 'yes':
                        tags.append('Spicy')
                    if review_tags.get('is_halal') == 'yes':
                        tags.append('Halal')
                    if review_tags.get('is_gluten_free') == 'yes':
                        tags.append('Gluten-Free')
                    if tags:
                        print(f"   Tags: {', '.join(tags)}")
                    
                    print(f"   {price_info}{wait_info}")
                    
                    if review.get('text'):
                        # Show first 100 characters of review text
                        text_preview = review['text'][:100] + ('...' if len(review['text']) > 100 else '')
                        print(f"   \"{text_preview}\"")
                
                if len(r['results']) > 5:
                    print(f"\n   ... and {len(r['results']) - 5} more")
                
                if r['current_page'] < r['total_pages']:
                    print(f"\n   📄 Page {r['current_page']}/{r['total_pages']}")
                    
        else:
            print(f"\n❌ {response.status}: {user['email']} - {query['description']}")
            print(f"   Response: {response.text}")

    # Build params from query
    params = query['params'].copy()
    
    # Convert place_id to string if present
    if 'place_id' in params:
        params['place_id'] = str(params['place_id'])
    if 'user_id' in params:
        params['user_id'] = str(params['user_id'])
    
    # Rename lng to long for API
    if 'lng' in params:
        params['long'] = params.pop('lng')

    rts.append(
        RT(
            method="get",
            url=f'{BASE_URL_FUNCTIONS}/reviews',
            params=params,
            timeout=30,
            headers=tu.auth_json_headers(user["token"]),
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=1)

# Extract all discovered reviews from successful responses
stats_added, stats_updated_unique = 0, 0
seen_review_ids = set()
for response in responses:
    if response.status_code == 200:
        r = response.json()
        if r and 'results' in r:
            for review_data in r['results']:
                _, was_added = tu.update_or_add_object(review_data, reviews, identifier_key='id')
                if was_added:
                    stats_added += 1
                elif review_data['id'] not in seen_review_ids:
                    stats_updated_unique += 1
                    seen_review_ids.add(review_data['id'])

print(f"\nLocal records: Updated {stats_updated_unique} and Added {stats_added} new. (total {len(reviews)})")

In [ ]:
# show reviews from local records. Show first 3 reviews with images
if False:
    print(f"\nShowing first 3 reviews with images from local records:")
    reviews_with_images = [r for r in reviews if r.get('images')]
    for review in reviews_with_images[:3]:
        print(f"- {review.get("meal_name", "N/A")} by {review['action_caller']['email']} ({len(review['images'])} images)")
        print(json.dumps(review, indent=2))
    

### Update reviews

In [ ]:
review_updates = [
    {
        "review_test_id": "avocado-sandwich-1",
        "action_caller": None,  # Will be determined dynamically
        "form_data_str": {
            "meal_name": "Avocado Sandwich - UPDATED",
            "rating": "5",
            "text": "Updated review: Even better than I remembered! The avocado is always fresh.",
            "price": "7500",
        },
        "form_data_files": {
            "add_images": [
                "img_local/generic-pizza-2.jpg",
            ],
        },
        "remove_image_ids": [],
    },
    {
        "review_test_id": "veggie-supreme-1",
        "action_caller": None,
        "form_data_str": {
            "rating": "5",
            "text": "Changed my mind - this is actually amazing for the price!",
        },
        "form_data_files": {
            "add_images": [],
        },
        "remove_image_ids": [],
    },
    {
        "review_test_id": "spicy-chicken-burger-1",
        "action_caller": None,
        "form_data_str": {
            "waiting_time_minutes": "20",
        },
        "form_data_files": {
            "add_images": [],
        },
        "remove_image_ids": [],
    },
]

rts = []

print(f"Running {len(review_updates)} updates...")
for upd in review_updates:
    # Get the user who owns this review
    if "action_caller" not in upd or upd["action_caller"] is None:
        user = tu.get_user_that_owns_review_with_test_id(upd['review_test_id'], reviews, users)
    else:
        user = upd["action_caller"]
    
    review = tu.get_review_from_test_id(upd['review_test_id'], reviews)
    
    if 'id' not in review:
        print(f"Review {upd['review_test_id']} doesn't have an ID yet, skipping update")
        continue

    form = FormData()

    # Add string form data
    if 'form_data_str' in upd:
        for field, value in upd['form_data_str'].items():
            form.add_field(field, str(value))

    # Add file form data
    if 'form_data_files' in upd and upd['form_data_files'].get('add_images'):
        for file_path in upd['form_data_files']['add_images']:
            form.add_field(
                'add_images',
                open(file_path, "rb"),
                filename=Path(file_path).name,
                content_type="image/jpeg"
            )

    # Add remove_image_ids if present
    if upd.get('remove_image_ids'):
        remove_ids_str = ','.join(str(img_id) for img_id in upd['remove_image_ids'])
        form.add_field('remove_image_ids', remove_ids_str)

    def callback(response: bu.CustomResponse, user=user, upd=upd, review=review):
        if response.status_code == 200:
            r = response.json()
            if r != None:
                print(f"{response.status}: {user['email']} updated review {upd['review_test_id']} (ID: {review['id']})")
                print(f"   Message: {r.get('message', 'Success')}")
            
        else:
            print(f"{response.status}: {user['email']} failed to update review {upd['review_test_id']}")
            if response.json() is not None:
                print(json.dumps(response.json(), indent=2))
            else:
                print(f"   Response: {response.text}")

    rts.append(
        RT(
            method="put",
            url=f"{BASE_URL_FUNCTIONS}/reviews/{review['id']}",
            data=form,
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=60,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False, max_concurrent=3)

### Delete Reviews

In [ ]:
# Select reviews to delete - typically low-rated or test reviews
reviews_to_delete = [
    {
        "action_caller": None, # Determined auto
        "review_test_id": "cheese-pizza-1",  # Pizza School Cheese Pizza (2 stars)
    },
    {
        "action_caller": None,
        "review_test_id": "italian-bmt-1",  # Subway Italian BMT (1 star)
    },
]

rts = []

for delete_query in reviews_to_delete:
    if "action_caller" not in delete_query or delete_query["action_caller"] is None:
        user = tu.get_user_that_owns_review_with_test_id(delete_query['review_test_id'], reviews, users)
    else:
        user = delete_query['action_caller']
    
    try:
        review = tu.get_review_from_test_id(delete_query['review_test_id'], reviews)
    except ValueError as e:
        print(f"Review {delete_query['review_test_id']} not found, skipping")
        continue
    
    if 'id' not in review:
        print(f"Review {delete_query['review_test_id']} doesn't have an ID yet, skipping delete")
        continue

    def callback(response: bu.CustomResponse, user=user, query=delete_query, review=review):
        if response.status_code == 204:
            print(f"{response.status}: {user['email']} deleted {query['review_test_id']} (ID: {review['id']})")
            
            # Remove from local reviews list
            reviews.remove(review)
            print(f"   Removed from local records")
        else:
            print(f"{response.status}: {user['email']} failed to delete {query['review_test_id']}")
            print(f"   Response: {response.text}")

    rts.append(
        RT(
            method="delete",
            url=f"{BASE_URL_FUNCTIONS}/reviews/{review['id']}",
            headers={
                "Authorization": f"Bearer {user['token']}",
            },
            timeout=30,
            callback=callback
        )
    )

responses = bu.run_async_requests(rts, verbose=False)

print(f"\nRemaining reviews in local list: {len(reviews)}")

## Swiping

### Interactive Recommendation Test
Run the cell below to fetch a feed, pick a meal, and swipe on it with debug info.

In [ ]:
import uuid

# Configuration
TEST_USER_EMAIL = "elias@t.com"
SESSION_ID = str(uuid.uuid4())

TEST_LAT = 36.373
TEST_LNG = 127.367

# Reset State
OPTION = "s"
feed_queue = []
current_meal = None
history_queue = []
last_swipe_debug = None

# Get user token
user = tu.get_user_from_email(TEST_USER_EMAIL, users)
if not user or 'token' not in user:
    print(f"User {TEST_USER_EMAIL} not found or not logged in. Please run setup cells first.")
else:
    print(f"Testing as {user['first_name']} {user['last_name']}")

In [ ]:
def fetch_feed(user, limit=3):
    url = f'{BASE_URL_FUNCTIONS}/users/me/feed'
    headers = tu.auth_json_headers(user['token'])
    params = {
        "limit": limit,
        "lat": TEST_LAT,
        "long": TEST_LNG,
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching feed: {response.status_code} {response.text}")
        return []

def send_swipe(user, meal_id, liked, session_id):
    url = f'{BASE_URL_FUNCTIONS}/swipes'
    headers = tu.auth_json_headers(user['token'])
    params = {"debug": True}
    data = {
        "meal_id": meal_id,
        "liked": liked,
        "session_id": session_id
    }
    response = requests.post(url, headers=headers, json=data, params=params)
    return response

def print_meal_card(meal):
    print(f"\n{'='*40}")
    print(f"🍱 NEXT MEAL: {meal['name']}")
    print(f"   Place: {meal.get('place_name', 'Unknown Place')}")
    
    # Cuisine
    cuisine = meal.get('cuisine') or meal.get('place_cuisine') or 'N/A'
    print(f"   Cuisine: {cuisine}")
    
    # Waiting Time
    wait_time = meal.get('avg_waiting_time') or meal.get('waiting_time_minutes')
    if wait_time:
        print(f"   Wait Time: ~{wait_time} min")
        
    # Distance
    distance = meal.get('distance_meters')
    if distance is not None:
        if distance >= 1000:
            distance_km = distance / 1000
            print(f"   Distance: {distance_km:.2f} km")
        else:
            print(f"   Distance: {distance:.0f} m")
    
    tags = meal.get('tags', {})
    active_tags = [k.replace('is_', '').title() for k, v in tags.items() if v == 'yes']
    negative_tags = [k.replace('is_', '').title() for k, v in tags.items() if v == 'no']
    
    if active_tags:
        print(f"   Tags (Yes): {', '.join(active_tags)}")
    if negative_tags:
        print(f"   Tags (No): {', '.join(negative_tags)}")
    
    print(f"   Price: {meal.get('avg_price') or meal.get('price') or 'N/A'}")
    print(f"   Rating: {meal.get('avg_rating', 'N/A')}")
    print(f"{'='*40}")

In [ ]:
# Interactive Manual Swipe
# Change OPTION to 'l', 'd', 's', or 'f' and run the cell.

try:
    OPTION # type: ignore
except NameError:
    OPTION = 's'  # Default option

# Configuration
PAGE_SIZE = 1
MIN_QUEUE_SIZE = 1

# Initialize Global State (Persists across cell runs)
if 'feed_queue' not in globals():
    feed_queue = []
if 'current_meal' not in globals():
    current_meal = None
if 'history_queue' not in globals():
    history_queue = []
if 'last_swipe_debug' not in globals():
    last_swipe_debug = None

# --- 1. Process Previous Action ---
if current_meal:
    if OPTION == 'l':
        print(f"👍 LIKING: {current_meal['name']}")
        resp = send_swipe(user, current_meal['id'], True, SESSION_ID)
        if resp.status_code in [200, 201]:
            print("   ✅ Swipe Recorded!")
            history_queue.append({'action': 'like', 'meal': current_meal})
            
            # Save debug info to global
            data = resp.json()
            if 'debug_preferences' in data:
                last_swipe_debug = data['debug_preferences']
            else:
                last_swipe_debug = None
        else:
            print(f"   ❌ Error: {resp.status_code} {resp.text}")
            last_swipe_debug = None

    elif OPTION == 'd':
        print(f"👎 DISLIKING: {current_meal['name']}")
        resp = send_swipe(user, current_meal['id'], False, SESSION_ID)
        if resp.status_code in [200, 201]:
            print("   ✅ Swipe Recorded!")
            history_queue.append({'action': 'dislike', 'meal': current_meal})
            
            # Save debug info to global
            data = resp.json()
            if 'debug_preferences' in data:
                last_swipe_debug = data['debug_preferences']
            else:
                last_swipe_debug = None
        else:
            print(f"   ❌ Error: {resp.status_code} {resp.text}")
            last_swipe_debug = None

    elif OPTION == 's':
        print(f"⏭️ SKIPPING: {current_meal['name']}")
        history_queue.append({'action': 'skip', 'meal': current_meal})
        last_swipe_debug = None
    
    elif OPTION == 'f': # type: ignore
            print("🧹 FLUSHING Queue")
            feed_queue = []
            current_meal = None
            last_swipe_debug = None
else:
    if OPTION == 'f':
            print("🧹 FLUSHING Queue")
            feed_queue = []
            last_swipe_debug = None

# --- 2. Refill Queue ---
if len(feed_queue) < MIN_QUEUE_SIZE:
    print(f"\n🔄 Queue low ({len(feed_queue)}), fetching {PAGE_SIZE} more...")
    new_meals = fetch_feed(user, limit=PAGE_SIZE)
    if new_meals:
        feed_queue.extend(new_meals)
        print(f"   Added {len(new_meals)} meals. Queue size: {len(feed_queue)}")
    else:
        print("   ⚠️ No more meals returned from feed.")

# --- 3. Pick Next Meal ---
if feed_queue:
    current_meal = feed_queue.pop(0)
    print_meal_card(current_meal)
else:
    current_meal = None
    print("\n⚠️ Feed is empty. Try flushing ('f') or check backend.")

In [ ]:
# l - like, d - dislike, s - skip, f - flush
OPTION = 'l'

In [ ]:
# Show history summary
if history_queue:
    print(f"\n📜 History ({len(history_queue)} items): {[h['action'][0].upper() + ':' + h['meal']['name'] for h in history_queue]}")

if 'last_swipe_debug' in globals() and last_swipe_debug:
    print("\n📊 Updated User Preferences (Debug):")
    # Print each top-level key on one line to save vertical space
    for key, value in last_swipe_debug.items():
        print(f"   {key}: {json.dumps(value)}")